<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Epidemiology 104</h1>
<h1>Lecture</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [3]:
from collections import Counter
from pprint import pprint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import networkx as nx

import watermark

from NetworkEpiModel import *

%load_ext watermark
%matplotlib inline

We start by print out the versions of the libraries we're using for future reference

In [4]:
%watermark -n -v -m -g -iv

json      2.0.9
numpy     1.18.1
pandas    1.0.1
networkx  2.4
watermark 2.0.2
autopep8  1.5
scipy     1.4.1
Mon Apr 27 2020 

CPython 3.7.3
IPython 6.2.1

compiler   : Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 19.4.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit
Git hash   : afc8d87254a0d7946a0eabde181f736a93d48b8e


Load default figure style

In [5]:
plt.style.use('./d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Start

In [7]:
G = nx.erdos_renyi_graph(100, p=1.)

In [8]:
netSIR = NetworkEpiModel(G)
netSIR.add_interaction('S', 'I', 'I', 0.2)
netSIR.add_spontaneous('I', 'R', 0.1)

In [9]:
print(netSIR)

Epidemic Model with 3 compartments and 2 transitions:

S + I = I 0.200000
I -> R 0.100000

R0=2.00


In [15]:
network = G

In [17]:
N = network.number_of_nodes()
timesteps = 10

In [54]:
population = np.zeros((timesteps, N), dtype='str')

In [56]:
comps = list(netSIR.transitions.nodes)

In [57]:
comps

['S', 'I', 'R']

In [58]:
time = np.arange(1, timesteps, 1, dtype='int')

In [59]:
susceptible = netSIR._get_susceptible()

In [60]:
susceptible

'S'

In [61]:
pos = {'S':0, 'I':1, 'R':2}

In [62]:
seeds = {10:'I'}

In [63]:
seeds

{10: 'I'}

In [64]:
i = 0

for node in range(N):
    if node in seeds:
        population[i, node] = seeds[node]
    else:
        population[i, node] = susceptible

In [66]:
G.nodes()

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99))

In [67]:
node_i = 0

In [68]:
i = 1

In [69]:
state_i = population[i-1, node_i]

In [70]:
state_i

'S'

In [72]:
trans = netSIR.transitions.edges(state_i, data=True)

In [73]:
trans

OutMultiEdgeDataView([('S', 'I', {'agent': 'I', 'rate': 0.2})])

In [74]:
node_i

0

In [78]:
list(G.neighbors(node_i))

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

In [6]:
?np.random.multinomial

In [7]:
kwargs = {'S':100000-1, 'I':1, 'R':0}

pos = {comp: i for i, comp in enumerate(kwargs)}
labels = [str(i) for i in range(len(pos))]

for comp in pos:
    labels[pos[comp]] = comp

counts = np.zeros(len(pos), dtype='float')

for comp in pos:
    counts[pos[comp]] = kwargs[comp]

N0 = counts.sum()


In [8]:
population=np.random.multinomial(N0, counts/N0)


In [9]:
order = np.arange(N0, dtype='int')
np.random.shuffle(order)


In [10]:
order

array([18650, 26770, 74666, ..., 65620, 72466,  8704])

In [11]:
class NetworkEpiModel(EpiModel):
    def __init__(self, N, p):
        self.G = nx.erdos_renyi_graph(N, p)
        super(NetworkEpiModel, self).__init__()
        
    #def add_interaction(self, source, target, agent, rate):
        
    def _get_agents(self, labels):
        agents = {}
        
        for edge in self.transitions.edges(data=True):
            node_i, node_j, data = edge
            if "agent" in data:
                agent = labels[data['agent']]
                node_i = labels[node_i]
                
                if agent not in agents:
                    agents[agent] = set()
                    
                agents[agent].add(node_i)
                
        return agents
        
    def integrate(self, timesteps, **kwargs):
        N = self.G.number_of_nodes()
        
        population = kwargs['population']
        labels = kwargs['labels']

        order = np.arange(N)
        np.random.shuffle(order)

        agents = self._get_agents(labels)

        for time in np.arange(timestemps):
            new_pop = population.copy()

            for node_i in order:
                if population[node_i] in agents:
                    for node_j in self.G.edges(node_i):
                        if population[node_j] in agents[node_i]:
                            prob = np.random.random()
                            #rate = 



            np.random.shuffle(order)

        
#         pos = {comp: i for i, comp in enumerate(kwargs)}
#         labels = []*len(pos)
        
#         for comp in pos:
#             labels[pos[comp]] = comp
        
#         counts = np.zeros(len(pos), dtype='float')
        
#         for comp in pos:
#             counts[pos[comp]] = kwargs[comp]
        
#         N0 = counts.sum()

#         if N != N0:
#             raise PopulationMismatch('The initial population distribution must match the number of nodes in the graph')
        
#         population=np.random.multinomial(N, counts/N0)

        
#     def integrate(self, timesteps, **kwargs):
#         """Numerically integrate the epidemic model"""
#         pos = {comp: i for i, comp in enumerate(kwargs)}
#         population=np.zeros(len(pos))
        
#         for comp in pos:
#             population[pos[comp]] = kwargs[comp]
        
#         time = np.arange(1, timesteps, 1)

#         self.values_ = pd.DataFrame(scipy.integrate.odeint(self._new_cases, population, time, args=(pos,)), columns=pos.keys(), index=time)


SyntaxError: invalid syntax (<ipython-input-11-52ed9ef8802d>, line 43)

In [ ]:
net = NetworkEpiModel(1000, 1)

In [ ]:
net.add_interaction('S', 'I', 'I', 0.2)
net.add_spontaneous('I', 'R', 0.1)

In [ ]:
net._get_agents({'S':0, 'I':1, 'R':2})

In [ ]:
net.transitions.edges('S', data=True)

<div style="width: 100%; overflow: hidden;">
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</div>